In [32]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import pymongo
import json
import numpy as np

In [33]:
df = pd.read_excel('statefloracatalogdata.xlsx', sheet_name='StateFlora')

In [34]:
df.columns

Index(['Type', 'Genus', 'Tube colour', 'Botanical name', 'Common name',
       'States', 'NW', 'LE', 'NU', 'GT', 'FR', 'EA', 'EP', 'NL', 'MU', 'YP',
       'SL', 'KI', 'SE', 'Height (m)', 'Spread (m)', 'Rain (mm)',
       'Soil texture', 'Soil pH', 'Frost', 'Flower colour', 'Flower (time)',
       'License', 'Attribution link', 'Attribution Label'],
      dtype='object')

In [35]:
df = df[['Type', 'Genus', 'Tube colour', 'Botanical name', 'Common name',
         'Height (m)', 'Spread (m)', 'Rain (mm)',
       'Soil texture', 'Soil pH', 'Frost', 'Flower colour', 'Flower (time)']]
df = df.dropna()

In [36]:
df['Tube colour'].unique()

array(['B', 'G', ' B'], dtype=object)

In [37]:
df = df[df['Rain (mm)'] != 'AQU']
df = df[df['Rain (mm)'] != '*']
df = df.reset_index(drop = True)
len(df)

677

In [38]:
#df.loc[df['Tube colour'] == ' B', 'Tube colour'] = 'B'
#df.loc[df['Height (m)'] == '0. 1', 'Height (m)'] = '0.1'
#df.loc[df['Spread (m)'] == '     3-5', 'Spread (m)'] = '3-5'
#df.loc[df['Soil texture'] == 'Lo.', 'Soil texture'] = 'Lo'
#df.loc[df['Soil pH']== 'AN/C', 'Soil pH'] = 'ANC'

In [39]:
df = df.drop_duplicates()

In [40]:
df.Type.unique()

array(['Trees and Shrubs', 'Groundcover', 'Climbers', 'Grasses',
       'Other Strap-leaved Plants', 'Bulbs and Lilies',
       'Aquatic and Riparian Zone Plants', 'Rushes and Sedges'],
      dtype=object)

In [47]:
df['init'] = np.nan
df.loc[df['Type'] == 'Trees and Shrubs', 'init'] = 'TS'
df.loc[df['Type'] == 'Strap-leaved Plants', 'init'] = 'SLP'
df.loc[df['Type'] == 'Bulbs and Lilies', 'init'] = 'BAL'
df.loc[df['Type'] == 'Aquatic and Riparian Zone Plants', 'init'] = 'AARZP'
df.loc[df['Type'] == 'Rushes and Sedges', 'init'] = 'RAS'
df.loc[df['Type'] == 'Groundcover', 'init'] = 'GRO'
df.loc[df['Type'] == 'Climbers', 'init'] = 'CL'
df.loc[df['Type'] == 'Grasses', 'init'] = 'GRA'

In [48]:
df['init'].unique()

array(['TS', 'GRO', 'CL', 'GRA', 'SLP', 'BAL', 'AARZP', 'RAS'],
      dtype=object)

In [49]:
df['searchQuery'] = ''
df = df.reset_index(drop = True)
sublist = ['Tube colour','Height (m)', 'Spread (m)', 'Rain (mm)', 'Soil texture',
       'Soil pH', 'Frost', 'Flower colour', 'Flower (time)']

for indx, row in df.iterrows():
    if row['Type'] == 'Other Strap-leaved Plants':
        df.loc[df.index == indx, 'Type'] = 'Strap-leaved Plants'
    for each in sublist:
        tmp = row[each]
        if type(tmp) == str and each != 'Soil texture':
            sub = tmp.replace(' ','*').replace('`','*').split('*')
            sub = [i for i in sub if i != '']
            df.loc[df.index == indx, each] = "".join(sub)
        
        if tmp == 'AN/C':
            df.loc[df.index == indx, each] = 'ANC'
            
        if each == 'Soil texture':
            new_list = tmp.replace('/',',').replace(' ',',').replace('.',',').split(',')
            new_list = [i for i in new_list if i != '']
            for i in range(len(new_list)):
                new_list[i] = str(df.iloc[indx]['init']) + '_' + new_list[i]
            
            df.loc[df.index == indx, 'searchQuery'] = ",".join(new_list)

In [50]:
dic = {}
col = list(df.columns)
for each in col:
    if each not in ['Botanical name', 'Common name', 'Genus', 'States', 'Type']:
        print(each, ": ", df[each].unique(), "\n")


Tube colour :  ['B' 'G'] 

Height (m) :  ['1-2' '1-3' '4-10' '1-5' '5-8' '2-4' '2-5' '2-3' '4-12' '2-7' '5-10'
 '1.5-2' '4-7' '3-5' '0.5-1.5' '0.2-0.8' '2.5-4' '4-8' '0.5-3' '7-20'
 '1-1.5' '2-6' '3-6' '6-10' '4-6' '1-2.5' '3-4' '0.5-2' '5-15' '2-2.5'
 '5-12' '8-20' '1.5-4' '6-20' '2' '10-20' '1' '0.5-1' '0.5' '3-8' '8-15'
 '2-8' '8-12' '3-7' '2-10' '4' '10-15' '6-15' '0.3' '1.5' '0.6-1' '13-25'
 '15-35' '10-40' '0.5-2.5' '0.6-1.5' '0.2-0.6' '0.2-1' '0.3-1' '0.2-0.4'
 '1.5-3' '1.5-2.2' '0.5-0.8' '0.5-1.2' '1-4' '15-25' '6-25' '10-30' '4-15'
 '12-20' '3-10' '7-15' '15-40' '6-12' '7-12' '5-20' '20-40' '6-8' '4-9'
 '5-18' '6-14' '15-50' '3-9' '10-12' '8-30' '20-35' '15-30' '8-18' '3-12'
 '1.5-5' '10-25' '5-14' '9-16' '20-30' '6-11' '1.2-1.5' '0.1-2' '0.2-1.5'
 '8-10' '05-1.5' '1-1.3' '0.3-0.6' '0.3-0.4' '5-7' '0.1-1' '0.8-1' '6-7'
 '0.2-0.5' '0.6' '1-1.2' '0.2' '0.1-0.3' '0.1-0.6' '0.3-0.5' '0.1'
 '0.1-0.2' '0.4' '0.1-0.5' '0.3-0.8' '0.3-0.7' '0.15-.3' '0.2-0.7'
 '0.4-0.7' '0.2-0.9' '0.7'

In [51]:
com_list = df['Common name']

In [52]:
dic = {}
for i in range(len(com_list)):
    if com_list[i] not in list(dic.keys()):
        dic[com_list[i]] = [i]
    else:
        dic[com_list[i]].append(i)    

In [53]:
new_list = []
for keys, values in dic.items():
    if len(values) > 1:
        #print(values)
        while len(values) > 1:
            new_list.append(values.pop())
        

In [54]:
new_list

[37,
 65,
 77,
 76,
 80,
 109,
 484,
 135,
 156,
 153,
 148,
 145,
 143,
 149,
 144,
 141,
 140,
 168,
 167,
 560,
 559,
 557,
 184,
 570,
 566,
 232,
 226,
 241,
 231,
 571,
 326,
 281,
 348,
 332,
 380,
 579,
 396,
 612,
 443,
 439,
 432,
 437,
 596,
 478,
 543,
 542,
 541,
 551,
 583,
 604]

In [55]:
df = df.drop(new_list)

In [58]:
df = df.reset_index(drop = True)
df

,Type,Genus,Tube colour,Botanical name,Common name,Height (m),Spread (m),Rain (mm),Soil texture,Soil pH,Frost,Flower colour,Flower (time),init,searchQuery
0,Trees and Shrubs,Acacia,B,acinacea,round-leaved wattle,1-2,1-2,300,"Sa,Lo,Cl",ANC,R,Y,"W,Sp",TS,"TS_Sa,TS_Lo,TS_Cl"
1,Trees and Shrubs,Acacia,B,anceps,Port Lincoln wattle,1-3,1-3,250,"Sa,Li",ANC,M,Y,"Sp,S",TS,"TS_Sa,TS_Li"
2,Trees and Shrubs,Acacia,B,aneura,mulga,4-10,3-6,150,"Sa,Lo,Li",ANC,R,Y,"W,Sp",TS,"TS_Sa,TS_Lo,TS_Li"
3,Trees and Shrubs,Acacia,B,argyrophylla,golden-grey mulga,1-5,2-6,350,"Sa,Lo,Li",ANC,R,Y,"W,Sp",TS,"TS_Sa,TS_Lo,TS_Li"
4,Trees and Shrubs,Acacia,B,baileyana var. purpurea,purple Cootamundra wattle,5-8,4-7,500,"Lo,Cl",ANC,R,Y,"W,Sp",TS,"TS_Lo,TS_Cl"
5,Trees and Shrubs,Acacia,B,brachybotrya,grey mulga,2-4,1.5-4,250,"Sa,Lo,Cl,Li",ANC,R,Y,"W,Sp",TS,"TS_Sa,TS_Lo,TS_Cl,TS_Li"
6,Trees and Shrubs,Acacia,B,calamifolia,wallowa,2-5,2-4,200,"Sa,Lo,Cl,Li",ANC,M,Y,"W,Sp",TS,"TS_Sa,TS_Lo,TS_Cl,TS_Li"
7,Trees and Shrubs,Acacia,B,continua,thorn wattle,1-2,1-2,300,"Sa,Lo,Cl",ANC,M,Y,"W,Sp",TS,"TS_Sa,TS_Lo,TS_Cl"
8,Trees and Shrubs,Acacia,B,cultriformis,knife-leaf wattle,2-4,2-4,450,"Lo,Cl",ANC,R,Y,"W,Sp",TS,"TS_Lo,TS_Cl"
9,Trees and Shrubs,Acacia,B,cupularis,coastal umbrella bush,2-3,2-3,200,"Sa,Lo,Li",ANC,M,Y,Sp,TS,"TS_Sa,TS_Lo,TS_Li"


In [63]:
df = df.drop(columns= ['init'])


KeyError: "['init'] not found in axis"

In [69]:
new_list = list(df.columns)
for i in range(len(new_list)):
    sub = new_list[i].split(" ")
    if len(sub) > 1:
        new_list[i] = "".join(sub)

In [70]:
new_list

['Type',
 'Genus',
 'Tubecolour',
 'Botanicalname',
 'Commonname',
 'Height(m)',
 'Spread(m)',
 'Rain(mm)',
 'Soiltexture',
 'SoilpH',
 'Frost',
 'Flowercolour',
 'Flower(time)',
 'searchQuery']

In [71]:
df.columns = new_list

In [72]:
df.head()

,Type,Genus,Tubecolour,Botanicalname,Commonname,Height(m),Spread(m),Rain(mm),Soiltexture,SoilpH,Frost,Flowercolour,Flower(time),searchQuery
0,Trees and Shrubs,Acacia,B,acinacea,round-leaved wattle,1-2,1-2,300,"Sa,Lo,Cl",ANC,R,Y,"W,Sp","TS_Sa,TS_Lo,TS_Cl"
1,Trees and Shrubs,Acacia,B,anceps,Port Lincoln wattle,1-3,1-3,250,"Sa,Li",ANC,M,Y,"Sp,S","TS_Sa,TS_Li"
2,Trees and Shrubs,Acacia,B,aneura,mulga,4-10,3-6,150,"Sa,Lo,Li",ANC,R,Y,"W,Sp","TS_Sa,TS_Lo,TS_Li"
3,Trees and Shrubs,Acacia,B,argyrophylla,golden-grey mulga,1-5,2-6,350,"Sa,Lo,Li",ANC,R,Y,"W,Sp","TS_Sa,TS_Lo,TS_Li"
4,Trees and Shrubs,Acacia,B,baileyana var. purpurea,purple Cootamundra wattle,5-8,4-7,500,"Lo,Cl",ANC,R,Y,"W,Sp","TS_Lo,TS_Cl"


In [76]:
col

Index(['Type', 'Genus', 'Tubecolour', 'Botanicalname', 'Commonname',
       'Height(m)', 'Spread(m)', 'Rain(mm)', 'Soiltexture', 'SoilpH', 'Frost',
       'Flowercolour', 'Flower(time)', 'searchQuery'],
      dtype='object')

In [78]:
tot = []
col = df.columns
for index, row in df.iterrows():
    result = {}
    for each in col:
        result[each] = row[each]
    
    tot.append(result)

In [79]:
with open('data.json', 'w') as f:
    json.dump(tot, f)

[{'Type': 'Trees and Shrubs',
  'Genus': 'Acacia ',
  'Tubecolour': 'B',
  'Botanicalname': 'acinacea ',
  'Commonname': 'round-leaved wattle',
  'Height(m)': '1-2',
  'Spread(m)': '1-2',
  'Rain(mm)': 300,
  'Soiltexture': 'Sa,Lo,Cl',
  'SoilpH': 'ANC',
  'Frost': 'R',
  'Flowercolour': 'Y',
  'Flower(time)': 'W,Sp',
  'searchQuery': 'TS_Sa,TS_Lo,TS_Cl'},
 {'Type': 'Trees and Shrubs',
  'Genus': 'Acacia ',
  'Tubecolour': 'B',
  'Botanicalname': 'anceps',
  'Commonname': 'Port Lincoln wattle',
  'Height(m)': '1-3',
  'Spread(m)': '1-3',
  'Rain(mm)': 250,
  'Soiltexture': 'Sa,Li',
  'SoilpH': 'ANC',
  'Frost': 'M',
  'Flowercolour': 'Y',
  'Flower(time)': 'Sp,S',
  'searchQuery': 'TS_Sa,TS_Li'},
 {'Type': 'Trees and Shrubs',
  'Genus': 'Acacia ',
  'Tubecolour': 'B',
  'Botanicalname': 'aneura',
  'Commonname': 'mulga',
  'Height(m)': '4-10',
  'Spread(m)': '3-6',
  'Rain(mm)': 150,
  'Soiltexture': 'Sa,Lo,Li',
  'SoilpH': 'ANC',
  'Frost': 'R',
  'Flowercolour': 'Y',
  'Flower(time)'

In [80]:
df.to_csv('all data.csv')